### Currently broken. Needs to be converted into python scripts utilising functions as part of custom packages.

In [ ]:
import os
import ntpath
import shutil
import sys
import argparse
from datetime import datetime
import random

import numpy as np
import cv2
from PIL import Image, ImageChops, ImageDraw, ImageOps, ImageFilter, ImageStat, ImageEnhance, ImageFile
from skimage import io, color, exposure
import matplotlib
import matplotlib.pyplot as plt
import glob
import math

from damage import no_damage, remove_quadrant, remove_hole, bullet_holes, graffiti, bend_vertical, damage_image
from utils import load_paths, load_files, scale_image, create_alpha, delete_background, to_png, dir_split, png_to_jpeg
import manipulate
import generate

In [ ]:
import yaml

with open("config.yaml", "r") as ymlfile:
    config = yaml.load(ymlfile, Loader=yaml.FullLoader)

# Directory names excluded from config.yaml to make use of .gitignore simpler
base_dir        = "Sign_Templates"
input_dir       = os.path.join(base_dir, "1_Input")
processed_dir   = os.path.join(base_dir, "2_Processed")
damaged_dir     = os.path.join(base_dir, "3_Damaged")
transformed_dir = os.path.join(base_dir, "4_Transformed")
manipulated_dir = "5_Manipulated"

bg_dir    = "Backgrounds"
final_dir = "SGTS_Dataset"  # SGTSD stands for "Synthetically Generated Traffic Sign Dataset"

In [ ]:
# Create the output directories if they don't exist already
if not os.path.exists(base_dir):
    os.mkdir(base_dir)
if os.path.exists(processed_dir):
    shutil.rmtree(processed_dir)
os.mkdir(processed_dir)

# Rescale images and make white backgrounds transparent
paths = load_files(input_dir)
for path in paths:
    img = scale_image(path, config['sign_width']) # Rescale the image

    # Remove the extension and save as a png
    _, filename = ntpath.split(path)
    name, _ = filename.rsplit('.', 1)
    save_path = os.path.join(processed_dir, name) + ".png"
    img.save(save_path)

    delete_background(save_path, save_path) # Overwrite the newly rescaled image

In [ ]:
# Used for SGTSD  TODO: Get rid of or fix this fiend, or at least its dodgy dependencies
damage_types = config['damage_types']

#TODO: Cmd line parameter for reusing pre-generated damaged images

# Remove any old output and recreate the output directory
reusable = config['reuse_damage']
if not reusable:
    shutil.rmtree(damaged_dir)
if not os.path.exists(damaged_dir):
    os.mkdir(damaged_dir)
    
    ii = 0
    processed = load_files(processed_dir)
    for image_path in processed:
        print("Damaging signs: " + str(float(ii) / float(len(processed)) * 100) + " %")
        labels_dir = os.path.join(final_dir, "Labels")
        damage_image(image_path, damaged_dir, labels_dir)
        ii += 1
    print("Damaging signs: " + str(100) + " %")
else:
    print("Reusing pre-existing damaged signs")

In [ ]:
# Loop through each folder in the input directory
for folder_path in load_paths(damaged_dir):
    # Retrieve the sign number to create a directory
    _,number = os.path.split(folder_path)
    save_dir = os.path.join(transformed_dir, number)
    
    for image_path in load_files(folder_path):
        manipulate.img_transform(image_path, save_dir)

In [ ]:
ImageFile.LOAD_TRUNCATED_IMAGES = True

for bg_folders in load_paths(bg_dir):
    to_png(bg_folders)

In [ ]:
for dirs in load_paths(bg_dir):
    sep = os.sep  # Directory separator: '/' or '\'

    if config['man_method'] == 'exposure':
        for background in load_paths(dirs):
            iniitial, subd, element = dir_split(background)
            title, extension = element.split('.')

            for signp in load_paths(transform_dir):
                for sign in load_paths(signp):
                    d,s,f,e = dir_split(sign)  # Eg. s = 4_Transformed_Images, f = 9, e = 9_BOTTOM_HOLE

                    exp_dir = manipulated_dir + sep
                    if (not os.path.exists(exp_dir + subd + sep + title + sep + "SIGN_" + f + sep + e)):
                        os.makedirs(exp_dir + subd + sep + title + sep + "SIGN_" + f + sep + e)
    else:
        for signp in load_paths(transform_dir):
            for sign in load_paths(signp):
                d,s,f,e = dir_split(sign)

                fade_dir = manipulated_dir + sep
                if (not os.path.exists(fade_dir + "SIGN_" + f + sep + e)):
                    os.makedirs(fade_dir + "SIGN_" + f + sep + e)

signs_paths = []
for p in load_paths(transform_dir):
    for d in load_paths(p):
        signs_paths += load_paths(d)

background_paths = []  # Load the paths of the background images into a single list

for subfolder in load_paths(bg_dir):
    background_paths += load_paths(subfolder)

if config['man_method'] == 'exposure':
    manipulate.exposure_manipulation(signs_paths, background_paths)
else:
    manipulate.fade_manipulation(signs_paths, background_paths)

In [ ]:
if config['man_method'] == 'exposure':
    manipulate.find_useful_signs(manipulated_dir)

In [ ]:
# Create the required directories in SGTSD
if (not os.path.exists(os.path.join(final_dir, "Images"))):
    # Numbered Version
    for sign in load_paths(input_dir):  # Dir. for each sign type
        head, tail = sign.split('.')
        name = dir_split(head)
        os.makedirs(os.path.join(final_dir, "Images", name[-1]))
        j = 0
        for dmg in range(len(damage_types)):  # Dir. for each damage type
            os.makedirs(os.path.join(final_dir, "Images", name[-1], name[-1] + "_" + str(j)))
            j = j + 1
    
    # Named Version (Outdated?)
#     for sign in load_paths(os.path.join("Traffic_Signs_Templates", "1_Input_Images")):  # Dir. for each sign type
#         head, tail = sign.split('.')
#         name = dir_split(head)
#         os.makedirs(os.path.join("SGTSD", "Images", name[-1]))
#         for dmg in load_paths(os.path.join("Traffic_Signs_Templates", "3_Damaged_Images")): #Dir. for each damage type
#             headD,tailD = dmg.split('.')
#             nameD = dir_split(headD)
#             os.makedirs(os.path.join("SGTSD", "Images", name[-1], nameD[-1]))

In [ ]:
# Create a README file  #TODO: Really necessary?
content = '''
-----------------------------------------------
|                     -*-                     |
|Synthetically Generated Traffic Sign Dataset |
|                     -*-                     |
-----------------------------------------------

This directory contains the generated training
set to be used for training a Convolutional
Neural Network (CNN).

It may be used for any detector desired and it
is not limited to a specific set of traffic
sign templates.
 

----------------------------------------------
Content
----------------------------------------------

The number of examples is based on the number:
->of traffic signs that were used as templates
->of damage types applied to images
->of transformations in image manipulations
->of the brightness variation values used
->of blending procedures


----------------------------------------------
Image format and naming
----------------------------------------------

The images created are of "jpg" format
with RGBA channels.

   XXX/XXX_YYY/XXX_YYY_ZZZ.jpg

(X) is used to distinguish the sign class, (Y)
is used to distinguish the damage type and (Z)
is used to indicate the example number.


----------------------------------------------
Additional information
----------------------------------------------

Contact Email:
   Original Code:
	   asterga@essex.ac.uk

   Adapted Damage Code:
      kristian.rados@student.curtin.edu.au
      seana.dale@student.curtin.edu.au


----------------------------------------------
'''

text_file = open(os.path.join(final_dir, "Readme_Images.txt"), "w")
text_file.write(content)
text_file.close()

In [ ]:
directories = generate.assigned_paths_list(manipulated_dir, bg_dir)
print("Files to be generated: " + str(len(directories)))

In [ ]:
#TODO: Something has gone wrong here onwards... final_directories is filled with empty elements

In [ ]:
final_directories = []  # Reformat list to have each sign and damage as their own dimensions
signs = load_paths(os.path.join("Traffic_Signs_Templates", "1_Input_Images"))
for i in signs:
    head, tail = ntpath.split(i)
    sign, extension = tail.split('.')  # Eg. sign == "9"

    sign_list = []  # List of damages, which are each list of signs
    for dmg in damage_types:  # Damage_types is from 'def damage_images:' cell
        sign_list.append(generate.list_for_sign_x(sign, dmg, directories))
    final_directories.append(sign_list)  # List of types -> lists of damages -> lists of signs

In [ ]:
# Generate and write the new data to its file
direct = os.path.join("SGTSD", "Images")
direct1 = os.path.join("SGTSD", "Labels")
folders = load_paths(direct)
n = []  # Numbers from the folder names for the signs
for folder in folders:
    head, tail = ntpath.split(folder)
    n.append(tail)

# Count how many signs there are; needed for the progress bar
total = 0
for signs in final_directories:
    for damages in signs:
        for dirs in damages:
            total += 1

count = 0
ii = 0
for signs in final_directories:  # Iterate through sign types
    jj = 0
    for damages in signs:  # Iterate through damage types
        # FIXME: For some reason secondary progress counter has broken from initial implementation
        print("Processed: " + str(float(count) / float(total) * 100) + " %")
        kk = 0

        label_filename = os.path.join(direct1, n[ii], n[ii] + "_" + str(ij) + ".txt")
        text_file = open(label_filename, "r")
        values = text_file.read()  # Retrieve the damage values created earlier in the damage_images function
        text_file = open(label_filename, "w")

        for dirs in damages:  # dirs == the foreground and background images for one generated sign
            filename = os.path.join(direct, n[ii], n[ii] + "_" + str(ij), n[ii] + "_" + str(ij) + "_" + str(kk))
            image = generate.new_data(dirs[0], dirs[1], text_file, filename, values) # Combining b.g. with sign f.g.
            cv2.imwrite(filename + ".png", image)

            count += 1
            kk += 1
        jj += 1
        text_file.close()
    ii += 1
print("Processed: " + str(100) + " %")  # FIXME: changing damage names broke something

In [ ]:
string = '''
-------------------------------------
BREAKDOWN OF FILES GENERATED BY CLASS
-------------------------------------
'''
total = 0
for i in range(len(final_directories)):
    current = 0
    for j in range(len(final_directories[i])):
        current = current + len(final_directories[i][j])
    s = "Generated " + str(current) + " examples for sign class " + str(i + 1)
    string = string + '\n' + s + '\n'
    total = total + current
string = string + '\n' + "TOTAL: " + str(total) + '\n' + "Generated on: " + datetime.now().strftime("%Y-%m-%d %H:%M") + '\n'
string = string + "-------------------------------------"
text_file = open(os.path.join("SGTSD", "generated_images_about.txt"), "w")
text_file.write(string)
text_file.close()

In [ ]:
dirs = direct = os.path.join(final_dir, "Images")
i = 1
for path in load_paths(dirs):
    print("Processed: " + str(float(i - 1) / float(len(final_directories)) * 100) + " %")
    for damage in load_paths(path):
        for image in load_paths(damage):
            if (image.endswith("png")):
                png_to_jpeg(image)
    i += 1
print("Processed: " + str(100) + " %")

In [ ]:
shutil.rmtree(manipulated_dir)

In [ ]:
shutil.rmtree(transformed_dir)
shutil.rmtree(damaged_dir)
shutil.rmtree(processed_dir)

In [ ]:
shutil.rmtree(final_dir)  # Be careful with this one